# Daily Returns Analysis (2020-2024)

This notebook analyzes daily returns for S&P 500, Gold, Bitcoin, Ethereum, and XRP from 2020 to 2024.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8-darkgrid')

print("Libraries imported successfully!")

## 1. Load the Data

In [ ]:
# Load the combined data file
df = pd.read_csv('all_assets_daily_returns_2020_2024.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

print(f"Data shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nDate range: {df.index.min()} to {df.index.max()}")
print(f"\nFirst few rows:")
df.head()

## 2. Summary Statistics

In [ ]:
# Calculate summary statistics for returns
return_columns = [col for col in df.columns if col.endswith('_Return')]
assets = [col.replace('_Return', '') for col in return_columns]

summary_stats = pd.DataFrame(index=['Mean', 'Std Dev', 'Min', 'Max', 'Skewness', 'Kurtosis', 'Sharpe Ratio'])

for asset in assets:
    returns = df[f'{asset}_Return'].dropna()
    summary_stats[asset] = [
        returns.mean(),
        returns.std(),
        returns.min(),
        returns.max(),
        returns.skew(),
        returns.kurtosis(),
        (returns.mean() / returns.std()) * np.sqrt(252)  # Annualized Sharpe Ratio
    ]

print("Summary Statistics for Daily Returns (%):\n")
summary_stats.round(4)

## 3. Correlation Analysis

In [ ]:
# Calculate correlation matrix
correlation_matrix = df[return_columns].corr()

# Create correlation heatmap
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, 
            mask=mask,
            annot=True, 
            fmt='.3f',
            cmap='RdBu_r', 
            center=0,
            square=True,
            linewidths=1,
            cbar_kws={"shrink": .8})
plt.title('Daily Returns Correlation Matrix (2020-2024)', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

# Display correlation matrix
print("\nCorrelation Matrix:")
correlation_matrix

## 4. Return Distributions

In [ ]:
# Plot return distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, asset in enumerate(assets):
    returns = df[f'{asset}_Return'].dropna()
    axes[i].hist(returns, bins=50, density=True, alpha=0.7, color='skyblue', edgecolor='black')
    axes[i].set_title(f'{asset} Daily Returns Distribution', fontsize=12)
    axes[i].set_xlabel('Daily Return (%)')
    axes[i].set_ylabel('Density')
    
    # Add normal distribution overlay
    from scipy import stats
    x = np.linspace(returns.min(), returns.max(), 100)
    axes[i].plot(x, stats.norm.pdf(x, returns.mean(), returns.std()), 'r-', linewidth=2, label='Normal')
    axes[i].legend()

# Remove empty subplot
fig.delaxes(axes[-1])
plt.tight_layout()
plt.show()

## 5. Cumulative Returns

In [ ]:
# Calculate cumulative returns
plt.figure(figsize=(14, 8))

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
for i, asset in enumerate(assets):
    cumulative_returns = (1 + df[f'{asset}_Return']/100).cumprod() - 1
    plt.plot(df.index, cumulative_returns * 100, label=asset, color=colors[i], linewidth=2)

plt.xlabel('Date', fontsize=12)
plt.ylabel('Cumulative Return (%)', fontsize=12)
plt.title('Cumulative Returns (2020-2024)', fontsize=16, pad=20)
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Display final cumulative returns
print("\nFinal Cumulative Returns:")
for asset in assets:
    final_return = ((1 + df[f'{asset}_Return']/100).cumprod() - 1).iloc[-1] * 100
    print(f"{asset}: {final_return:.2f}%")

## 6. Rolling Statistics

In [ ]:
# Calculate rolling statistics
window = 30  # 30-day rolling window

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Rolling volatility
for asset in assets:
    rolling_vol = df[f'{asset}_Return'].rolling(window=window).std() * np.sqrt(252)
    ax1.plot(df.index, rolling_vol, label=f'{asset}', linewidth=1.5)

ax1.set_title(f'{window}-Day Rolling Annualized Volatility', fontsize=14)
ax1.set_ylabel('Volatility (%)', fontsize=12)
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# Rolling correlation with S&P 500
for asset in assets[1:]:  # Skip S&P 500 itself
    rolling_corr = df['SP500_Return'].rolling(window=window).corr(df[f'{asset}_Return'])
    ax2.plot(df.index, rolling_corr, label=f'{asset} vs S&P 500', linewidth=1.5)

ax2.set_title(f'{window}-Day Rolling Correlation with S&P 500', fontsize=14)
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Correlation', fontsize=12)
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(-1, 1)

plt.tight_layout()
plt.show()

## 7. Risk-Return Analysis

In [ ]:
# Risk-Return scatter plot
plt.figure(figsize=(10, 8))

# Calculate annualized returns and volatility
annual_returns = []
annual_vols = []

for asset in assets:
    returns = df[f'{asset}_Return'].dropna()
    annual_return = returns.mean() * 252
    annual_vol = returns.std() * np.sqrt(252)
    annual_returns.append(annual_return)
    annual_vols.append(annual_vol)
    
    plt.scatter(annual_vol, annual_return, s=200, alpha=0.7)
    plt.annotate(asset, (annual_vol, annual_return), 
                xytext=(5, 5), textcoords='offset points', fontsize=12)

plt.xlabel('Annualized Volatility (%)', fontsize=12)
plt.ylabel('Annualized Return (%)', fontsize=12)
plt.title('Risk-Return Profile (2020-2024)', fontsize=16, pad=20)
plt.grid(True, alpha=0.3)

# Add efficient frontier line (simplified)
plt.plot([0, max(annual_vols)*1.1], [0, max(annual_vols)*1.1*0.5], 
         'k--', alpha=0.3, label='Risk-Return Trade-off')
plt.legend()
plt.tight_layout()
plt.show()

## 8. Monthly Performance Heatmap

In [ ]:
# Create monthly returns heatmap for each asset
fig, axes = plt.subplots(3, 2, figsize=(16, 12))
axes = axes.flatten()

for i, asset in enumerate(assets):
    # Calculate monthly returns
    monthly_returns = df[f'{asset}_Return'].resample('M').apply(lambda x: ((1 + x/100).prod() - 1) * 100)
    
    # Pivot to create year-month matrix
    monthly_pivot = monthly_returns.to_frame('return')
    monthly_pivot['Year'] = monthly_pivot.index.year
    monthly_pivot['Month'] = monthly_pivot.index.month
    monthly_matrix = monthly_pivot.pivot(index='Year', columns='Month', values='return')
    
    # Create heatmap
    sns.heatmap(monthly_matrix, 
                annot=True, 
                fmt='.1f',
                cmap='RdYlGn', 
                center=0,
                ax=axes[i],
                cbar_kws={'label': 'Return (%)'})
    axes[i].set_title(f'{asset} Monthly Returns (%)', fontsize=12)
    axes[i].set_xlabel('Month')
    axes[i].set_ylabel('Year')

# Remove empty subplot
fig.delaxes(axes[-1])
plt.tight_layout()
plt.show()

## 9. Export Analysis Results

In [ ]:
# Export summary statistics
summary_stats.to_csv('analysis_summary_statistics.csv')
print("Summary statistics exported to 'analysis_summary_statistics.csv'")

# Export correlation matrix
correlation_matrix.to_csv('analysis_correlation_matrix.csv')
print("Correlation matrix exported to 'analysis_correlation_matrix.csv'")

# Create a comprehensive report
report = {
    'analysis_date': datetime.now().strftime('%Y-%m-%d'),
    'data_period': {
        'start': str(df.index.min().date()),
        'end': str(df.index.max().date()),
        'total_days': len(df)
    },
    'assets_analyzed': assets,
    'key_findings': {
        'best_performer': assets[annual_returns.index(max(annual_returns))],
        'highest_volatility': assets[annual_vols.index(max(annual_vols))],
        'lowest_volatility': assets[annual_vols.index(min(annual_vols))],
        'highest_correlation_pair': 'See correlation matrix for details'
    }
}

import json
with open('analysis_report.json', 'w') as f:
    json.dump(report, f, indent=2)
    
print("\nAnalysis report exported to 'analysis_report.json'")
print("\nAnalysis complete!")